In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

In [2]:
url = "https://www.statcan.gc.ca/en/subjects/standard/sccai/2011/scountry-desc"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

In [3]:
table = soup.find("table")
rows = table.find_all("tr")
header_rows = table.find_all("tr")[:2]

In [4]:
level_1 = ['Current and Historical Countries and Areas of Interest'] * 5 + ['Period of Validity'] * 2 + ['Remarks']
level_2 = ['Code', 'Countries and Areas of Interest', 'Num-3', 'Alpha-2', 'Alpha-3', 'Start', 'End', '']
headers = pd.MultiIndex.from_tuples(list(zip(level_1, level_2)))
headers

MultiIndex([('Current and Historical Countries and Areas of Interest', ...),
            ('Current and Historical Countries and Areas of Interest', ...),
            ('Current and Historical Countries and Areas of Interest', ...),
            ('Current and Historical Countries and Areas of Interest', ...),
            ('Current and Historical Countries and Areas of Interest', ...),
            (                                    'Period of Validity', ...),
            (                                    'Period of Validity', ...),
            (                                               'Remarks', ...)],
           )

In [5]:
data = []
data_rows = table.find_all("tr")[2:]

for row in data_rows:
    cols = row.find_all('td')
    if cols and (len(cols) == 8 or len(cols) == 7):
        row_data = [ele.get_text(strip=True) for ele in cols]
        if len(row_data) == 7:
            row_data.append('')
        
        data.append(row_data)
data = [row for row in data if not 'Code:five-digit numerical code' in row[0]]

df = pd.DataFrame(data, columns=headers)
display(df)

Current and Historical Countries and Areas of Interest  \
                                                      Code   
0                                          Afghanistan       
1                                        Åland Islands       
2                                              Albania       
3                                              Algeria       
4                                       American Samoa       
..                                                 ...       
346                                         Yugoslavia       
347                                         Yugoslavia       
348                                              Zaire       
349                                             Zambia       
350                                           Zimbabwe       

                                                          Period of Validity  \
    Countries and Areas of Interest Num-3 Alpha-2 Alpha-3              Start   
0                               004    AF     AFG    1970                      
1                               248    AX     ALA    2004                      
2                               008    AL     ALB    1970                      
3                               012    DZ     DZA    1970                      
4                               016    AS     ASM    1970                      
..                              ...   ...     ...     ...                ...   
346                             891    YU     YUG    1992               2003   
347                             890    YU     YUG    1991               1992   
348                             180    ZR     ZAR    1971               1997   
349                             894    ZM     ZMB    1970                      
350                             716    ZW     ZWE    1980                      

                                                       Remarks  
                                                   End          
0                                                               
1                            Formerly part of Finland.          
2                                                               
3                                                               
4                                                               
..                                                 ...     ...  
346  New numeric code for Yugoslavia and in 2003 Yu...          
347  Divided into Bosnia and Herzegovina, the Forme...          
348  Now known as the Democratic Republic of the Co...          
349                                                             
350               Formerly known as Southern Rhodesia.          

[351 rows x 8 columns]

In [6]:
df.to_csv(os.path.join(os.getcwd(), 'Datasets', 'Raw', 'country_codes.csv'))